In [ ]:
import cv2

In [ ]:
print(cv2.__version__)

## Demo

In [ ]:
import pandas as pd
import math

# Training Site
# water_col = [4750, 3849, 3061, 4026, 1975]
# water_row = [3899, 3907, 3259, 3498, 5805]

# land_col = [1211, 1308, 1298, 2212, 2262]
# land_row = [2102, 2168, 2659, 2361, 3324]

# veg_col = [6021, 6465, 4293, 6291, 4069]
# veg_row = [3128, 3534, 5175, 5408, 6524]

# Training Site
water_col = [329, 660, 716, 1089, 954]
water_row = [996, 695, 1233, 914, 773]

land_col = [457, 814, 764, 824, 627]
land_row = [139, 335, 831, 1165, 1194]

veg_col = [533, 489, 737, 676, 778]
veg_row = [337, 329, 119, 961, 1070]

water = pd.DataFrame()
land = pd.DataFrame()
veg = pd.DataFrame()

water['column'] = water_col
water['row'] = water_row

land['column'] = land_col
land['row'] = land_row

veg['column'] = veg_col
veg['row'] = veg_row


In [ ]:
# B G R
# b = cv2.imread('blue.PNG', 0)
# g = cv2.imread('green.PNG', 0)
# r = cv2.imread('red.PNG', 0)

b = cv2.imread('landsat8/LC09_L2SP_116050_20230224_20230226_02_T1_SR_B2.TIF', 0)
g = cv2.imread('landsat8/LC09_L2SP_116050_20230224_20230226_02_T1_SR_B3.TIF', 0)
r = cv2.imread('landsat8/LC09_L2SP_116050_20230224_20230226_02_T1_SR_B4.TIF', 0)

image_merge = cv2.merge([b,g,r])

#calculate the percent of original dimensions
scale_percent = 20

width = int(image_merge.shape[1] * scale_percent / 100)
height = int(image_merge.shape[0] * scale_percent / 100)

# dsize
dsize = (width, height)
output = cv2.resize(image_merge, dsize)
output_grey = cv2.resize(b, dsize)

b = cv2.resize(b, dsize)
g = cv2.resize(g, dsize)
r = cv2.resize(r, dsize)

cv2.imwrite("rgb-resized-20.tif", output);
cv2.imwrite("b-resized-20.tif", output);
cv2.imwrite("g-resized-20.tif", output);
cv2.imwrite("r-resized-20.tif", output);

In [ ]:

image_merge = output

# get mean vector
water_b_mean = b[water['row'], water['column']].mean()
water_g_mean = g[water['row'], water['column']].mean()
water_r_mean = r[water['row'], water['column']].mean()

land_b_mean = b[land['row'], land['column']].mean()
land_g_mean = g[land['row'], land['column']].mean()
land_r_mean = r[land['row'], land['column']].mean()

veg_b_mean = b[veg['row'], veg['column']].mean()
veg_g_mean = g[veg['row'], veg['column']].mean()
veg_r_mean = r[veg['row'], veg['column']].mean()

print(water_b_mean, water_g_mean, water_r_mean)
print(land_b_mean, land_g_mean, land_r_mean)
print(veg_b_mean, veg_g_mean, veg_r_mean)



# grab the image dimensions
h = image_merge.shape[0]
w = image_merge.shape[1]

# loop over the image
for y in range(0, h):
    for x in range(0, w):
       
        if b[y,x] == 0 and g[y,x] == 0 and r[y,x] == 0:
            output[y, x] = [0, 0, 0]
        
        # minimum distance classifier
        dw = math.sqrt( (b[y,x] - water_b_mean)**2 + (g[y,x] - water_g_mean)**2 + (r[y,x] - water_r_mean)**2)
        dl = math.sqrt( (b[y,x] - land_b_mean)**2 + (g[y,x] - land_g_mean)**2 + (r[y,x] - land_r_mean)**2)
        dv = math.sqrt( (b[y,x] - veg_b_mean)**2 + (g[y,x] - veg_g_mean)**2 + (r[y,x] - veg_r_mean)**2)
        
        # water area, blue, 255
        if dw < dl and dw < dv :
            output[y, x] = [255, 0, 0]
            output_grey[y, x] = 255
        
        # land area, red, 167
        elif dl < dw and dl < dv :
            output[y, x] = [0, 0, 255]
            output_grey[y, x] = 167
        
        # vegetation area, green, 50
        elif dv < dw and dv < dl :
            output[y, x] = [0, 255, 0]
            output_grey[y, x] = 50
        
        # unclassified, black, 0
        else:
            output[y, x] = [255, 255, 255]
            output_grey[y, x] = 0
            
        print(y, x, b[y,x], g[y,x], r[y,x])
        
        

In [ ]:
# save result
cv2.imwrite("rgb-classified.tif", output);
cv2.imwrite("grey-classified.tif", output_grey);

In [ ]:
# view pixels from training if correctly classified
# water 255
# land 167
# veg 50

print("water", output_grey[water['row'], water['column']])

In [ ]:
print("land", output_grey[land['row'], land['column']])

In [ ]:
print("vegetation", output_grey[veg['row'], veg['column']])